In [8]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [9]:
train_X=pd.read_csv("pca_train.csv")
train_Y=pd.read_csv("train_label.csv").label
test_X=pd.read_csv("pca_test.csv")

In [10]:
x_train,x_val,y_train,y_val=train_test_split(
    train_X,train_Y,random_state=777
)

In [11]:
scaler=StandardScaler()
train_X_scale=scaler.fit_transform(x_train)
val_X_scale=scaler.transform(x_val)
test_X_scale=scaler.transform(test_X)

In [12]:
train_X_tensor=torch.FloatTensor(train_X_scale)
val_X_tensor=torch.FloatTensor(val_X_scale)
test_X_tensor=torch.FloatTensor(test_X_scale)

C:\Users\bjw01\AppData\Local\Temp\ipykernel_19148\1027207712.py:1: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  train_X_tensor=torch.FloatTensor(train_X_scale)


In [13]:
train_Y_tensor=torch.LongTensor(y_train)

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        drop_prob=0.3
        self.fc1=nn.Linear(256,198,bias=True)
        self.fc2=nn.Linear(198,198,bias=True)
        self.fc3=nn.Linear(198,198,bias=True)

        self.fc4=nn.Linear(198,198)
        self.relu=torch.nn.Sigmoid()
        self.dropout=torch.nn.Dropout(p=drop_prob)

        for m in self.modules():
            if isinstance(m,nn.Linear):
                nn.init.xavier_uniform_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.zero_()

    def forward(self,x):
        x=self.fc1(x)
        x=self.fc2(x)
        x=self.fc3(x)
        x=self.fc4(x)
    
        return x

In [15]:
import torch.optim as optim
net=Net()
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=0.01)

In [19]:
from sklearn.metrics import accuracy_score
epoch=2000

for i in range(2000):
    optimizer.zero_grad()
    outputs=net(train_X_tensor)
    loss=criterion(outputs,train_Y_tensor)
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print(i, loss.item())

0 5.830221652984619
100 4.257011890411377
200 3.276442766189575
300 2.8519458770751953
400 2.6318633556365967
500 2.4909234046936035
600 2.3833367824554443
700 2.293797492980957
800 2.216186046600342
900 2.147749662399292
1000 2.0870909690856934
1100 2.0333211421966553
1200 1.9854575395584106
1300 1.9425201416015625
1400 1.9036799669265747
1500 1.868277668952942
1600 1.8357831239700317
1700 1.8057646751403809
1800 1.777876377105713
1900 1.7518455982208252


In [23]:
with torch.no_grad():
    hypothesis=net(train_X_tensor)
    print(hypothesis)

tensor([[-0.7449, -2.1479, -0.4236,  ...,  1.0348, -0.4850,  1.1967],
        [-0.5611,  1.1202,  0.1329,  ..., -1.9181, -1.6917,  0.5332],
        [-3.1346, -1.1330, -3.1533,  ..., -1.4276,  0.5291, -0.2146],
        ...,
        [ 1.9555,  1.5535, -2.7393,  ..., -0.9010, -1.0183, -0.7856],
        [ 1.5329, -0.1411, -1.0327,  ..., -1.5405, -0.8009, -0.2452],
        [ 0.6483, -0.7290, -1.6017,  ..., -0.9854, -0.3363,  1.8604]])
